# TSUMUGIに必要なアノテーション情報を整理する

In [1]:
# Move up to top directory
import os
from pathlib import Path

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir('../')

print(os.getcwd())

/mnt/e/TSUMUGI/notebooks/notebools-web
/mnt/e/TSUMUGI


In [2]:
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import csv
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import json

In [3]:
# def write_jsonl(data_dict: dict[list[str]], file_path: str | Path):
#     # JSON Lines形式で保存
#     with open(file_path, "w", encoding="utf-8") as f:
#         for key, values in data_dict.items():
#             json_line = {key: values}
#             f.write(json.dumps(json_line) + "\n")

In [4]:
# def read_jsonl(file_path: str | Path) -> dict[list[str]]:
#     # JSON Lines形式で読み込み
#     data_dict = defaultdict(list)
#     with open(file_path, "r", encoding="utf-8") as f:
#         for line in f:
#             entry = json.loads(line)
#             for key, values in entry.items():
#                 data_dict[key].extend(values)
#     return dict(data_dict)


In [5]:
path_data = Path("data", "statistical_filtered.csv")

In [6]:
data = pd.read_csv(path_data)
print(len(data))
# Release 22.0: 54059

54059


/tmp/ipykernel_41768/1284382592.py:1: DtypeWarning: Columns (25,60) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path_data)


In [7]:
Path("data/annotation").mkdir(exist_ok=True, parents=True)

In [8]:
import numpy as np

threshold = 0.0001

# 条件リスト
conditions = [
    (data["sex_effect_p_value"] < threshold) & (data["female_ko_effect_p_value"] < threshold) & (data["male_ko_effect_p_value"] > threshold),
    (data["sex_effect_p_value"] < threshold) & (data["male_ko_effect_p_value"] < threshold) & (data["female_ko_effect_p_value"] > threshold)
]

# 条件に対応する値
choices = ["♀", "♂"]

# np.selectで列を設定
data["sexdual_dimorphism"] = np.select(conditions, choices, default=None)

# 結果を確認
print(data["sexdual_dimorphism"].value_counts())


sexdual_dimorphism
♂    4915
♀    4146
Name: count, dtype: int64


In [9]:
filtered_data = data[data["sexdual_dimorphism"].notna()]
filtered_data[["p_value", "sexdual_dimorphism", "effect_size", "genotype_effect_parameter_estimate", "female_ko_parameter_estimate", "male_ko_parameter_estimate"]].head(10)

,p_value,sexdual_dimorphism,effect_size,genotype_effect_parameter_estimate,female_ko_parameter_estimate,male_ko_parameter_estimate
7,2.473221e-01,♂,-0.775659,-9.880265,-9.880265,-52.709552
9,9.323751e-02,♂,0.637815,-407.079557,-434.647221,1362.539526
20,5.407602e-05,♀,0.246786,12.886045,12.886045,2.852505
24,3.072703e-06,♀,-1.242673,-74.143434,-91.529079,-61.885708
27,2.315925e-13,♀,2.053221,1.424075,1.421448,0.715908
28,2.623063e-01,♂,1.258237,0.286702,0.286702,1.119338
38,3.010097e-01,♂,0.942703,0.364327,0.364327,1.333887
50,7.644770e-01,♂,1.428011,0.047221,0.047221,0.976487
51,3.243832e-02,♂,1.024187,450.500694,450.500694,1026.044584
54,4.852500e-06,♂,0.063404,1.479369,1.109724,1.709209


## marker_symbol と mp_term_name の対応付け

In [10]:
data_select = data[['marker_symbol', 'mp_term_name', 'zygosity', "sexdual_dimorphism"]].drop_duplicates()
# data_select = data[['marker_symbol', 'marker_accession_id', 'mp_term_name', 'mp_term_id']].drop_duplicates()
data_select

,marker_symbol,mp_term_name,zygosity,sexdual_dimorphism
0,Uap1,abnormal lens morphology,heterozygote,None
1,6430548M08Rik,enlarged kidney,homozygote,None
2,Stac,abnormal locomotor behavior,homozygote,None
3,Slc24a4,enlarged thymus,heterozygote,None
4,Uhrf2,female infertility,homozygote,None
...,...,...,...,...
54048,Lrrk1,abnormal joint morphology,homozygote,None
54051,Rtl5,abnormal kidney morphology,hemizygote,None
54052,Adcy3,increased total body fat amount,heterozygote,♀
54055,Alg3,"preweaning lethality, complete penetrance",homozygote,None


In [11]:
data_select["zygosity"].value_counts()

zygosity
homozygote      31206
heterozygote     9725
hemizygote        519
Name: count, dtype: int64

In [12]:
data_dict = defaultdict(list)
for index, row in data_select.iterrows():
    if row['zygosity'] == 'homozygote':
        annotate = "Homo"
    elif row['zygosity'] == 'heterozygote':
        annotate = "Hetero"
    else:
        annotate = "Hemi"

    if row['sexdual_dimorphism'] == "♀":
        annotate = ", ".join([annotate, '♀'])
    if row['sexdual_dimorphism'] == "♂":
        annotate = ", ".join([annotate, '♂'])

    data_dict[row['marker_symbol']].append(f"{row['mp_term_name']} ({annotate})")   

for key in data_dict.keys():
    data_dict[key].sort()

print(data_dict["Rhd"])

['abnormal skin condition (Homo)', 'decreased circulating HDL cholesterol level (Homo, ♂)', 'decreased circulating alkaline phosphatase level (Homo, ♀)', 'decreased circulating cholesterol level (Homo, ♂)', 'decreased circulating free fatty acids level (Homo)', 'decreased hemoglobin content (Homo, ♂)', 'decreased mean corpuscular hemoglobin (Homo)', 'decreased mean corpuscular hemoglobin concentration (Homo)', 'decreased mean corpuscular volume (Homo)', 'increased exploration in new environment (Homo)']


In [13]:
file_path = "data/annotation/symbol_mptermname.json"
json.dump(data_dict, open(file_path, "w"), indent=4, sort_keys=True)


In [14]:
%%bash

head -n 10 data/annotation/symbol_mptermname.json

{
    "0610010K14Rik": [
        "embryonic lethality prior to organogenesis (Homo)",
        "embryonic lethality prior to tooth bud stage (Homo)",
        "increased startle reflex (Hetero, \u2642)",
        "preweaning lethality, complete penetrance (Homo)"
    ],


    "0610040J01Rik": [
        "abnormal heart morphology (Homo)",
        "abnormal spleen morphology (Homo)",


## Connect mp_term_name to IMPC Phenotype URL

In [15]:
data_select = data[['mp_term_id', 'mp_term_name']].drop_duplicates()
# data_select = data[['marker_symbol', 'marker_accession_id', 'mp_term_name', 'mp_term_id']].drop_duplicates()
data_select

,mp_term_id,mp_term_name
0,MP:0001303,abnormal lens morphology
1,MP:0003068,enlarged kidney
2,MP:0001392,abnormal locomotor behavior
3,MP:0000709,enlarged thymus
4,MP:0001926,female infertility
...,...,...
46204,MP:0004818,increased skeletal muscle mass
47247,MP:0005407,hyperalgesia
48663,MP:0003956,abnormal body size
51033,MP:0009477,small cecum


In [16]:
data_dict_url = dict()
for index, row in data_select.iterrows():
    mp_tern_id = row['mp_term_id']
    impc_url = f"https://www.mousephenotype.org/data/phenotypes/{mp_tern_id}"
    mp_term_name = row['mp_term_name']
    data_dict_url[mp_term_name] = impc_url

print(data_dict_url["small lymph nodes"])

https://www.mousephenotype.org/data/phenotypes/MP:0002217


In [17]:
with open('data/annotation/mptermname_phenotypeurl.tsv', 'w') as f:
    for term, url in data_dict_url.items():
        f.write(f"{term}\t{url}\n")

In [18]:
%%bash

head -n 3 data/annotation/mptermname_phenotypeurl.tsv
wc -l data/annotation/mptermname_phenotypeurl.tsv # Release 22.0: 664 

abnormal lens morphology	https://www.mousephenotype.org/data/phenotypes/MP:0001303
enlarged kidney	https://www.mousephenotype.org/data/phenotypes/MP:0003068
abnormal locomotor behavior	https://www.mousephenotype.org/data/phenotypes/MP:0001392
664 data/annotation/mptermname_phenotypeurl.tsv


## Connect marker_symbol to accession_id

In [19]:
data_select = data[['marker_symbol', 'marker_accession_id']].drop_duplicates()
# data_select = data[['marker_symbol', 'marker_accession_id', 'mp_term_name', 'mp_term_id']].drop_duplicates()
data_select

,marker_symbol,marker_accession_id
0,Uap1,MGI:1334459
1,6430548M08Rik,MGI:2443793
2,Stac,MGI:1201400
3,Slc24a4,MGI:2447362
4,Uhrf2,MGI:1923718
...,...,...
53904,Tm9sf1,MGI:1921390
53963,Eci3,MGI:1916373
53987,Stambp,MGI:1917777
54006,Wdr45b,MGI:1914090


In [20]:
data_dict = dict()
for index, row in data_select.iterrows():
    data_dict[row['marker_symbol']] = row['marker_accession_id']

In [21]:
data_dict["Ncam1"]

'MGI:97281'

In [22]:
json.dump(data_dict, open("data/annotation/symbol_mgiid.json", "w"), indent=4, sort_keys=True)
Path("data/annotation/symbol_mgiid.tsv").write_text("\n".join([f"{k}\t{v}" for k, v in data_dict.items()]))

140147

In [23]:
%%bash
head -n 3 data/annotation/symbol_mgiid.json
head -n 3 data/annotation/symbol_mgiid.tsv

{
    "0610010K14Rik": "MGI:1915609",
    "0610040J01Rik": "MGI:1923511",
Uap1	MGI:1334459
6430548M08Rik	MGI:2443793
Stac	MGI:1201400
